In [2]:
!pip install pandas numpy py-algorand-sdk plotly ipython

   ---------------------------------------- 0.0/19.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.1 MB 435.7 kB/s eta 0:00:44
   ---------------------------------------- 0.1/19.1 MB 980.4 kB/s eta 0:00:20
   - -------------------------------------- 0.8/19.1 MB 5.7 MB/s eta 0:00:04
   ---- ----------------------------------- 2.1/19.1 MB 11.1 MB/s eta 0:00:02
   ------- -------------------------------- 3.6/19.1 MB 15.3 MB/s eta 0:00:02
   ---------- ----------------------------- 5.2/19.1 MB 18.3 MB/s eta 0:00:01
   ------------- -------------------------- 6.3/19.1 MB 19.3 MB/s eta 0:00:01
   --------------- ------------------------ 7.2/19.1 MB 19.9 MB/s eta 0:00:01
   ---------------- ----------------------- 7.9/19.1 MB 18.6 MB/s eta 0:00:01
   ------------------ --------------------- 9.0/19.1 MB 19.1 MB/s eta 0:00:01
   --------------------- ------------------ 10.2/19.1 MB 19.7 MB/s eta 0:00:01



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from algosdk.v2client import indexer
import plotly.graph_objects as go
from IPython.display import display, clear_output
import logging
from typing import Dict, List, Optional
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

In [20]:
@dataclass
class WhaleMetrics:
    total_balance: float = 0
    avg_balance: float = 0
    std_balance: float = 0
    total_volume: float = 0
    active_whales: int = 0
    avg_tx_size: float = 0
    concentration: float = 0

class WhaleMonitor:
    def __init__(self, 
                 min_balance: float = 100_000, 
                 window_size: int = 24,
                 indexer_address: str = "https://mainnet-idx.algonode.cloud",
                 indexer_token: str = ""):
        headers = {
            "X-API-Key": indexer_token if indexer_token else "B3SU4KcVKi94Jap2VXkK83xx38bsv95K5UZm2lab"
        }
        self.client = indexer.IndexerClient(
            indexer_token="", 
            indexer_address=indexer_address,
            headers=headers
        )
        self.min_balance = min_balance * 1_000_000
        self.window_size = window_size
        self.transactions_df = pd.DataFrame()
        self._setup_logging()
        
    def _setup_logging(self):
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)

    def _process_transaction(self, tx: dict) -> dict:
        processed = {
            'id': tx.get('id', ''),
            'round': tx.get('confirmed-round', 0),
            'timestamp': datetime.fromtimestamp(tx.get('round-time', 0)),
            'sender': tx.get('sender', ''),
            'amount': 0,
            'receiver': ''
        }
        
        if 'payment-transaction' in tx:
            payment = tx['payment-transaction']
            processed.update({
                'amount': payment.get('amount', 0),
                'receiver': payment.get('receiver', '')
            })
        
        return processed

    def fetch_transactions(self, start_time: datetime, end_time: datetime) -> pd.DataFrame:
        try:
            response = self.client.search_transactions(
                min_amount=10_000_000,
                limit=500,
                start_time=start_time.isoformat() + 'Z',
                end_time=end_time.isoformat() + 'Z'
            )
            
            if not response.get("transactions"):
                return pd.DataFrame()
            
            processed_txs = [self._process_transaction(tx) for tx in response["transactions"]]
            return pd.DataFrame(processed_txs)
            
        except Exception as e:
            self.logger.error(f"Error fetching transactions: {e}")
            return pd.DataFrame()

    def update_transactions_window(self, new_transactions: pd.DataFrame):
        if new_transactions.empty:
            return
            
        self.transactions_df = pd.concat(
            [self.transactions_df, new_transactions],
            ignore_index=True
        )
        
        if not self.transactions_df.empty:
            self.transactions_df = self.transactions_df.drop_duplicates(subset=['id'])
            cutoff_time = datetime.utcnow() - timedelta(hours=self.window_size)
            self.transactions_df = self.transactions_df[
                self.transactions_df['timestamp'] >= cutoff_time
            ]

    def calculate_metrics(self) -> WhaleMetrics:
        if self.transactions_df.empty:
            return WhaleMetrics()

        whale_txs = self.transactions_df[self.transactions_df['amount'] >= self.min_balance]
        if whale_txs.empty:
            return WhaleMetrics()

        metrics = WhaleMetrics(
            total_balance=whale_txs['amount'].sum() / 1_000_000,
            avg_balance=whale_txs['amount'].mean() / 1_000_000,
            std_balance=whale_txs['amount'].std() / 1_000_000,
            total_volume=whale_txs['amount'].sum() / 1_000_000,
            active_whales=len(whale_txs['sender'].unique()),
            avg_tx_size=whale_txs['amount'].mean() / 1_000_000,
            concentration=(whale_txs.groupby('sender')['amount'].sum() ** 2).sum() / (whale_txs['amount'].sum() ** 2)
        )
        
        return metrics

    def plot_metrics(self, metrics: WhaleMetrics):
        fig = go.Figure()
        
        fig.add_trace(go.Indicator(
            mode="number",
            value=metrics.total_balance,
            title="Total Balance (ALGO)",
            domain={'row': 0, 'column': 0}
        ))
        
        fig.add_trace(go.Indicator(
            mode="number",
            value=metrics.active_whales,
            title="Active Whales",
            domain={'row': 0, 'column': 1}
        ))
        
        fig.add_trace(go.Indicator(
            mode="gauge+number",
            value=metrics.concentration * 100,
            title="Concentration Index (%)",
            domain={'row': 1, 'column': 0},
            gauge={'axis': {'range': [0, 100]}}
        ))
        
        fig.update_layout(
            grid={'rows': 2, 'columns': 2},
            height=600
        )
        
        return fig
    

monitor = WhaleMonitor(min_balance=10_000)
end_time = datetime.utcnow()
start_time = end_time - timedelta(minutes=1)
transactions = monitor.fetch_transactions(start_time, end_time)
monitor.update_transactions_window(transactions)
metrics = monitor.calculate_metrics()
fig = monitor.plot_metrics(metrics)
fig.show()